# Settings & Variables

In [ ]:
!pip install ddi-fw==0.0.239
!pip install ddi-fw[llm,ml]
# !pip install  openai langchain
# # !pip install  openai langchain sentence_transformers -q
# !pip install chromadb -q
# !pip install unstructured -q
# ! pip install langchain-community

# If Google Cloud Platform will be used for mlflow and storage, please upload GCP credentials

In [ ]:
from google.colab import files

# Upload your service account key JSON file
# uploaded = files.upload()

# List the uploaded files to verify
# uploaded

In [ ]:
# import os

# # Set the path to the uploaded service account key file
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path_of_credentials"

# # Verify the environment variable is set
# print("GOOGLE_APPLICATION_CREDENTIALS set to:", os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))

# Project Setup

In [ ]:
!pip freeze

In [ ]:
from google.colab import userdata

In [ ]:
token = userdata.get("GITHUB_API_KEY")
username = userdata.get("GITHUB_USER_NAME")
repo = 'enhancing-ddi-classification-via-textual-arguments'
path = '/content/'+ repo

# Cloning GitHub Repository

In [ ]:
!git clone https://{token}@github.com/{username}/{repo}

In [ ]:
%cd {repo}

In [ ]:
!pwd

In [ ]:
%ls

In [ ]:
import os

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [ ]:
key_of_embedding_store = '0295501b-e673-43e9-af91-7d06ec21cb7d'
dataset_id = f'kivancbayraktar/{key_of_embedding_store}'
ddi_mdl_experiment_1 = f'{path}/experiment_configurations/ddi-mdl/experiment_1.json'
ddi_mdl_experiment_2 = f'{path}/experiment_configurations/ddi-mdl/experiment_2.json'
ddi_mdl_experiment_3 = f'{path}/experiment_configurations/ddi-mdl/experiment_3.json'

mdf_sa_ddi_experiment_1=f'{path}/experiment_configurations/mdf-sa-ddi/experiment_1.json'
mdf_sa_ddi_experiment_2_tui=f'{path}/experiment_configurations/mdf-sa-ddi/experiment_2_only_tui.json'
mdf_sa_ddi_experiment_2_cui=f'{path}/experiment_configurations/mdf-sa-ddi/experiment_2_only_cui.json'
mdf_sa_ddi_experiment_2_entities=f'{path}/experiment_configurations/mdf-sa-ddi/experiment_2_only_entities.json'
mdf_sa_ddi_experiment_3 = f'{path}/experiment_configurations/mdf-sa-ddi/experiment_3.json'

# Set experiment_configuration_file to run
experiment_configuration_file = ddi_mdl_experiment_1

In [ ]:
# !kaggle datasets download -d kivancbayraktar/0295501b-e673-43e9-af91-7d06ec21cb7d
!kaggle datasets download -d {dataset_id}

In [ ]:
from ddi_fw.utils import ZipHelper,Py7ZipHelper
import os
zip_helper = ZipHelper()
zip_helper.extract_single_file(file_path= f'{path}/{key_of_embedding_store}.zip', output_path=f'{path}/chroma_db')
os.remove(f'{path}/{key_of_embedding_store}.zip')

In [ ]:
import json
with open(experiment_configuration_file, 'r') as file:
    experiment_config  = json.load(file)

In [ ]:
experiment_config

In [ ]:
import chromadb

vector_db = chromadb.PersistentClient(
                    path="chroma_db")
vector_db.list_collections()
# column = 'description'
# collection = vector_db.get_collection("0295501b-e673-43e9-af91-7d06ec21cb7d_sentence")
# dictionary = collection.get(include=['embeddings', 'metadatas'], where= {"type": {"$eq": f'{column}'}})

In [ ]:
import tensorflow as tf
import numpy as np
import random

tf.random.set_seed(1)
np.random.seed(1)
random.seed(1)
# tf.random.set_seed(42)
# np.random.seed(42)
# random.seed(42)

In [ ]:
from ddi_fw.pipeline import MultiPipeline

In [ ]:
mp = MultiPipeline(experiment_configuration_file).build().run()

In [ ]:
print(f'name\t accuracy\tf-score\tprecision\trecall')
for key in mp.results().keys():
    for m in mp.results()[key].metric_dict:
      accuracy = mp.results()[key].metric_dict[m].accuracy
      macro_f_score = mp.results()[key].metric_dict[m].f1_score['macro']
      macro_precision = mp.results()[key].metric_dict[m].precision['macro']
      macro_recall = mp.results()[key].metric_dict[m].recall['macro']
      print(f'{m}: {accuracy}\t{macro_f_score}\t{macro_precision}\t{macro_recall}')